# Import Libraries

In [169]:
# import libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch 
import random
import gpytorch
from tqdm import trange
from botorch.models import SingleTaskGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qExpectedImprovement
from gpytorch.kernels import MaternKernel
from botorch.optim import optimize_acqf
import torch.optim as optim
from botorch.fit import fit_gpytorch_mll
from sklearn import preprocessing

# Load the needed dataset 

In [188]:
# load the AgNP dataset
data = pd.read_csv('datasets/AgNP_dataset.csv')
# merge rows that have the same values 
data = data.groupby(data.columns.tolist()).size().reset_index().rename(columns={0:'count'})
# for each of the rows with the same values merge them and take the average of the loss column
data = data.groupby(['QAgNO3(%)', 'Qpva(%)', 'Qtsc(%)', 'Qseed(%)', 'Qtot(uL/min)'], as_index=False).mean().reset_index()
# drop the count column 
data = data.drop(columns=['count', 'index'])

idxs = len(data)
# put the last column into the y variable
y = data.iloc[:, -1].values
# put the rest of the columns into the X variable
X = data.iloc[:, :-1].values

# standardize the X data
scalerX = preprocessing.StandardScaler().fit(X)
X = scalerX.transform(X)


X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

print(data.shape)



(164, 6)


# Initialize the surrogate model, kernel, and acquisition function

In [189]:
randSample = random.sample(range(0, idxs), 5)
X = X.clone()[randSample]
y = y.clone()[randSample]

bestX = X[torch.argmax(y)]
bestY = y.max()

# define the model
model = SingleTaskGP(X, y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)

# fit the model
fit_gpytorch_mll(mll)

# define the acquisition function use Expected Improvement
EI = qExpectedImprovement(model, best_f=bestY)

C:\Users\stan\AppData\Local\Temp\ipykernel_42968\4126337738.py:9: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(X, y)
c:\Users\stan\miniconda3\envs\research\Lib\site-packages\botorch\models\utils\assorted.py:174: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
c:\Users\stan\miniconda3\envs\research\Lib\site-packages\botorch\models\utils\assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([0.3896]), std = tensor([0.2139])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


# Optimize the model using BO 

In [190]:
optimizationCycles = 100
initialExp = 10

# optimize the acquisition function using Bayesian Optimization with botorch
bounds = torch.stack([torch.zeros(X.shape[1]), torch.ones(X.shape[1])])

for i in trange(optimizationCycles):
    model = SingleTaskGP(X, y)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    EI = qExpectedImprovement(model, best_f=bestY)

    # optimize the acquisition function
    new_X, _ = optimize_acqf(acq_function=EI, bounds=bounds, q=1, num_restarts=5, raw_samples=initialExp)
    
    # update the X and y values
    X = torch.cat((X, new_X))
    y = torch.cat((y, new_y))

#     # update the model
#     model = SingleTaskGP(X, y)
    
#     mll = ExactMarginalLogLikelihood(model.likelihood, model)
#     fit_gpytorch_mll(mll)

#     # update the acquisition function
#     EI = qExpectedImprovement(model, best_f=y.max())

#     # update the bounds
#     bounds = torch.stack([torch.zeros(X.shape[1]), torch.ones(X.shape[1])])

# # plot the results as y value vs index of the sample as a scatter plot
# plt.scatter(range(0, len(y)), y)
# plt.show()


y


  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\stan\AppData\Local\Temp\ipykernel_42968\3301798387.py:8: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(X, y)
c:\Users\stan\miniconda3\envs\research\Lib\site-packages\botorch\models\utils\assorted.py:174: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
c:\Users\stan\miniconda3\envs\research\Lib\site-packages\botorch\models\utils\assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([0.3896]), std = tensor([0.2139])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)
  0%|          | 0/100 [00:00<?, ?it/s]


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)